In [34]:
!pip install pandas matplotlib seaborn scipy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Carga del Dataset Limpio

En esta sección importamos la librería **pandas** y cargamos el dataset que ya ha sido previamente limpiado y procesado. 

**Archivo**: `cleaned_dataset.csv`  
**Contenido**: Datos preparados para el análisis de correlaciones

> **Nota**: Este dataset ha pasado por un proceso de limpieza previo, eliminando valores nulos, duplicados y normalizando los datos según sea necesario.

In [35]:
import pandas as pd
cleanDatatest = pd.read_csv("cleaned_dataset.csv")

# Inspección de la Estructura del Dataset

Utilizamos el método `.info()` para obtener información general sobre el dataset.

## Información que obtenemos:

- **Número total de filas y columnas**
- **Tipos de datos de cada columna**
- **Valores no nulos por columna**
- **Uso de memoria del DataFrame**

> **Importancia**: Esta información es crucial para entender la estructura de nuestros datos antes de proceder con el análisis de correlaciones.

In [36]:
cleanDatatest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986 entries, 0 to 4985
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              4986 non-null   object 
 1   turno                  4986 non-null   object 
 2   operador_id            4986 non-null   object 
 3   maquina_id             4986 non-null   object 
 4   producto_id            4986 non-null   object 
 5   temperatura            4986 non-null   float64
 6   vibración              4986 non-null   float64
 7   humedad                4986 non-null   float64
 8   tiempo_ciclo           4986 non-null   float64
 9   fallo_detectado        4986 non-null   object 
 10  tipo_fallo             495 non-null    object 
 11  cantidad_producida     4986 non-null   int64  
 12  unidades_defectuosas   4986 non-null   int64  
 13  eficiencia_porcentual  4986 non-null   float64
 14  consumo_energia        4986 non-null   float64
 15  para

# Análisis de Correlación Simple: Vibración vs Consumo de Energía

En esta celda realizamos un análisis básico de correlación entre dos variables específicas:

| Variable | Tipo | Descripción |
|----------|------|-------------|
| **Vibración** | Independiente | Variable que puede influir |
| **Consumo de energía** | Dependiente | Variable que queremos predecir |

## Proceso de análisis:

1. **Cálculo**: Coeficiente de correlación de Pearson
2. **Interpretación** basada en rangos:
   - **Correlación fuerte negativa**: < -0.5
   - **Correlación moderada negativa**: -0.5 a -0.3
   - **Correlación débil negativa**: -0.3 a 0

> **Objetivo**: Entender si existe una relación lineal entre estas dos variables clave.

In [37]:
# Calculate Pearson correlation coefficient
correlation = cleanDatatest['vibración'].corr(cleanDatatest['consumo_energia'])
print(f"Correlation coefficient: {correlation}")

# A negative correlation (closer to -1) indicates inverse relationship
if correlation < -0.5:
    print("Strong negative correlation - as one increases, the other decreases")
elif correlation < -0.3:
    print("Moderate negative correlation")
elif correlation < 0:
    print("Weak negative correlation")

Correlation coefficient: -0.006799881052127923
Weak negative correlation


# Definición de Funciones para Análisis Avanzado de Correlaciones

Esta celda contiene la definición de un **conjunto completo de funciones especializadas** para el análisis de correlaciones.

## Funciones incluidas:

| # | Función | Propósito |
|---|---------|-----------|
| 1 | `analyze_correlations()` | Función principal para análisis completo de correlaciones |
| 2 | `interpret_correlation()` | Convierte valores numéricos en interpretaciones cualitativas |
| 3 | `plot_correlation_heatmap()` | Genera visualizaciones tipo mapa de calor |
| 4 | `find_best_predictors()` | Identifica las mejores variables predictivas |

---

> **Beneficio**: Estas funciones proporcionan un **toolkit completo** para realizar análisis de correlaciones de manera sistemática y profesional, con capacidades tanto para análisis específicos como exploratorios generales.

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_correlations(df, target_column=None, threshold=0.3):
    """
    Analyze correlations between all numeric columns in the dataset
    
    Parameters:
    df: pandas DataFrame
    target_column: string, name of target variable (if None, analyzes all pairs)
    threshold: float, minimum correlation threshold to consider significant
    
    Returns:
    correlation_results: dict with correlation analysis
    """
    
    # Select only numeric columns
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f"Numeric columns found: {len(numeric_columns)}")
    print(f"Columns: {numeric_columns}\n")
    
    # Calculate correlation matrix
    correlation_matrix = df[numeric_columns].corr()
    
    results = {
        'correlation_matrix': correlation_matrix,
        'strong_correlations': [],
        'predictive_candidates': {}
    }
    
    if target_column:
        # Analyze correlations with specific target variable
        if target_column not in numeric_columns:
            print(f"Error: {target_column} is not a numeric column")
            return results
            
        target_correlations = correlation_matrix[target_column].drop(target_column)
        target_correlations = target_correlations.sort_values(key=abs, ascending=False)
        
        print(f"=== CORRELATIONS WITH {target_column.upper()} ===")
        print("-" * 50)
        
        for column, correlation in target_correlations.items():
            interpretation = interpret_correlation(correlation)
            print(f"{column:20s}: {correlation:6.3f} - {interpretation}")
            
            if abs(correlation) >= threshold:
                results['predictive_candidates'][column] = {
                    'correlation': correlation,
                    'interpretation': interpretation
                }
        
        results['target_correlations'] = target_correlations
        
    else:
        # Analyze all correlation pairs
        print("=== ALL SIGNIFICANT CORRELATIONS ===")
        print("-" * 50)
        
        # Get upper triangle of correlation matrix (avoid duplicates)
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)
        upper_triangle = correlation_matrix.where(mask)
        
        # Find correlations above threshold
        strong_corr_pairs = []
        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                corr_value = correlation_matrix.iloc[i, j]
                if abs(corr_value) >= threshold:
                    col1 = correlation_matrix.columns[i]
                    col2 = correlation_matrix.columns[j]
                    interpretation = interpret_correlation(corr_value)
                    
                    strong_corr_pairs.append({
                        'variable1': col1,
                        'variable2': col2,
                        'correlation': corr_value,
                        'interpretation': interpretation
                    })
        
        # Sort by absolute correlation value
        strong_corr_pairs.sort(key=lambda x: abs(x['correlation']), reverse=True)
        
        for pair in strong_corr_pairs:
            print(f"{pair['variable1']:15s} <-> {pair['variable2']:15s}: "
                  f"{pair['correlation']:6.3f} - {pair['interpretation']}")
        
        results['strong_correlations'] = strong_corr_pairs
    
    return results

def interpret_correlation(correlation):
    """Interpret correlation coefficient value"""
    abs_corr = abs(correlation)
    direction = "positive" if correlation > 0 else "negative"
    
    if abs_corr >= 0.8:
        strength = "Very strong"
    elif abs_corr >= 0.6:
        strength = "Strong"
    elif abs_corr >= 0.4:
        strength = "Moderate"
    elif abs_corr >= 0.2:
        strength = "Weak"
    else:
        strength = "Very weak"
    
    return f"{strength} {direction}"

def plot_correlation_heatmap(correlation_matrix, figsize=(12, 10)):
    """Create a correlation heatmap"""
    plt.figure(figsize=figsize)
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    
    # Generate heatmap
    sns.heatmap(correlation_matrix, 
                mask=mask,
                annot=True, 
                cmap='RdBu_r', 
                center=0,
                square=True,
                fmt='.2f',
                cbar_kws={"shrink": .8})
    
    plt.title('Correlation Matrix Heatmap')
    plt.tight_layout()
    plt.show()

def find_best_predictors(df, target_column, top_n=5):
    """Find the best predictor variables for a target column"""
    results = analyze_correlations(df, target_column)
    
    if 'target_correlations' not in results:
        return None
    
    target_correlations = results['target_correlations']
    
    print(f"\n=== TOP {top_n} PREDICTORS FOR {target_column.upper()} ===")
    print("-" * 50)
    
    top_predictors = target_correlations.head(top_n)
    
    for i, (column, correlation) in enumerate(top_predictors.items(), 1):
        interpretation = interpret_correlation(correlation)
        predictive_power = "High" if abs(correlation) > 0.6 else "Medium" if abs(correlation) > 0.3 else "Low"
        
        print(f"{i}. {column}")
        print(f"   Correlation: {correlation:.3f}")
        print(f"   Relationship: {interpretation}")
        print(f"   Predictive Power: {predictive_power}")
        print()
    
    return top_predictors

# Example usage with your dataset:
# Assuming your DataFrame is called 'testing'

# Method 1: Analyze correlations with a specific target (e.g., consumo_energia)
# results = analyze_correlations(testing, target_column='consumo_energia', threshold=0.3)

# Method 2: Find all significant correlations in the dataset
# results = analyze_correlations(testing, threshold=0.3)

# Method 3: Find best predictors for a specific target
# best_predictors = find_best_predictors(testing, 'consumo_energia', top_n=5)

# Method 4: Create correlation heatmap
# correlation_matrix = testing.select_dtypes(include=[np.number]).corr()
# plot_correlation_heatmap(correlation_matrix)

# Method 5: Quick correlation check (your original approach)
# correlation = testing['vibración'].corr(testing['consumo_energia'])
# print(f"Correlation coefficient: {correlation}")
# print(f"Interpretation: {interpret_correlation(correlation)}")

# Aplicación Práctica: Análisis de Paradas Imprevistas

En esta sección aplicamos las funciones definidas anteriormente para realizar un **análisis completo** centrado en la variable `paradas_imprevistas`.

## Análisis realizados:

### 1. **Correlaciones con variable objetivo** (`results`)
- Identifica qué variables se correlacionan con paradas imprevistas
- Utiliza un umbral de **0.3** para considerar correlaciones significativas

### 2. **Mejores predictores** (`best_predictors`)
- Encuentra las **5 variables** con mayor correlación
- Proporciona información detallada sobre cada predictor

### 3. **Análisis general** (`all_correlations`)
- Identifica **todas las correlaciones fuertes** en el dataset completo
- Utiliza un umbral más estricto de **0.4**

---

## Variables almacenadas:

| Variable | Contenido |
|----------|-----------|
| `results` | Análisis de correlaciones con la variable objetivo |
| `best_predictors` | Serie con los 5 mejores predictores ordenados |
| `all_correlations` | Diccionario con todas las correlaciones significativas |

> **Enfoque múltiple**: Permite entender tanto las relaciones específicas con nuestra variable de interés como el panorama general de correlaciones en el dataset.

In [39]:
# Find variables that predict energy consumption
results = analyze_correlations(cleanDatatest, target_column='paradas_imprevistas', threshold=0.3)

# Find the top 5 predictors for energy consumption
best_predictors = find_best_predictors(cleanDatatest, 'paradas_imprevistas', top_n=5)

# Find all strong correlations in your dataset
all_correlations = analyze_correlations(cleanDatatest, threshold=0.4)

Numeric columns found: 10
Columns: ['temperatura', 'vibración', 'humedad', 'tiempo_ciclo', 'cantidad_producida', 'unidades_defectuosas', 'eficiencia_porcentual', 'consumo_energia', 'paradas_programadas', 'paradas_imprevistas']

=== CORRELATIONS WITH PARADAS_IMPREVISTAS ===
--------------------------------------------------
cantidad_producida  : -0.018 - Very weak negative
unidades_defectuosas: -0.018 - Very weak negative
humedad             :  0.013 - Very weak positive
tiempo_ciclo        : -0.006 - Very weak negative
consumo_energia     : -0.005 - Very weak negative
temperatura         : -0.003 - Very weak negative
eficiencia_porcentual:  0.003 - Very weak positive
vibración           : -0.003 - Very weak negative
paradas_programadas :  0.001 - Very weak positive
Numeric columns found: 10
Columns: ['temperatura', 'vibración', 'humedad', 'tiempo_ciclo', 'cantidad_producida', 'unidades_defectuosas', 'eficiencia_porcentual', 'consumo_energia', 'paradas_programadas', 'paradas_imprevistas